In [19]:
import numpy as np
import torch 
import torch.nn as nn
import torch.autograd.function as F
import torch.utils.data as Data
torch.__version__

'1.1.0'

In [21]:
BATCH_SIZE = 64

In [8]:
def sample_data_1():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + rand.randn(count, 2) * [[5.0, 1.0]]
def sample_data_2():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + (rand.randn(count, 2) * [[5.0, 1.0]]).dot(
        [[np.sqrt(2) / 2, np.sqrt(2) / 2], [-np.sqrt(2) / 2, np.sqrt(2) / 2]])

In [6]:
torch.cuda.is_available()

False

In [5]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
data1 = np.asarray(sample_data_1(), dtype=np.float32)
data2 = np.asarray(sample_data_2(), dtype=np.float32)

In [15]:
data1_tensor = torch.from_numpy(data1).to(device)
data2_tensor = torch.from_numpy(data2).to(device)

In [32]:
dset1 = Data.TensorDataset(data1_tensor)
dset2 = Data.TensorDataset(data2_tensor)

In [22]:
dset1_loader = Data.DataLoader(
    dataset=dset1,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
)

dset2_loader = Data.DataLoader(
    dataset=dset2,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
)

In [43]:
for idx, [batch] in enumerate(dset1_loader):
    print(batch.size())
    break
    

torch.Size([64, 2])


In [3]:
class SimpleVAE(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x